In [ ]:
!pip install transformers torch

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import pipeline, BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

# Predefined dataset
dataset = [
    # Human Resource
    {"text": "I want to avail the services of Human Resource.", "label": 0},
    {"text": "I need assistance from the Human Resource department.", "label": 0},
    {"text": "Can I get support from HR?", "label": 0},
    {"text": "I want to access HR services.", "label": 0},
    {"text": "How do I get help from Human Resources?", "label": 0},
    {"text": "I want to contact HR for assistance.", "label": 0},
    {"text": "How can I use the HR services?", "label": 0},
    {"text": "I want to speak with someone from Human Resources.", "label": 0},
    {"text": "I need to use HR resources for my task.", "label": 0},
    {"text": "Can HR help me with my request?", "label": 0},

    # Financial Management
    {"text": "I want to avail the services of Financial Management.", "label": 1},
    {"text": "Can I get help from the Financial Management team?", "label": 1},
    {"text": "I need financial management support.", "label": 1},
    {"text": "How can I access Financial Management services?", "label": 1},
    {"text": "I want to consult the Financial Management department.", "label": 1},
    {"text": "How do I use Financial Management services?", "label": 1},
    {"text": "I need assistance with financial management.", "label": 1},
    {"text": "Can I avail financial management services for my project?", "label": 1},
    {"text": "I would like to access the services of the financial team.", "label": 1},
    {"text": "Can Financial Management help me with budgeting?", "label": 1},

    # Supply Chain Management
    {"text": "I want to avail the services of Supply Chain Management.", "label": 2},
    {"text": "Can I get support from the Supply Chain team?", "label": 2},
    {"text": "How can I access the services of Supply Chain Management?", "label": 2},
    {"text": "I need help with supply chain management services.", "label": 2},
    {"text": "How do I get assistance from the Supply Chain department?", "label": 2},
    {"text": "I want to use supply chain services.", "label": 2},
    {"text": "I need help with my supply chain management.", "label": 2},
    {"text": "Can I avail services from the Supply Chain team?", "label": 2},
    {"text": "I want to talk to someone from the Supply Chain department.", "label": 2},
    {"text": "How do I contact the Supply Chain team?", "label": 2},

    # Project Management
    {"text": "I want to avail the services of Project Management.", "label": 3},
    {"text": "How can I access Project Management services?", "label": 3},
    {"text": "I need assistance with project management.", "label": 3},
    {"text": "Can I get help from the Project Management department?", "label": 3},
    {"text": "I want to consult the Project Management team.", "label": 3},
    {"text": "How do I access the services of Project Management?", "label": 3},
    {"text": "I need to avail services from the Project Management team.", "label": 3},
    {"text": "Can Project Management help me with my task?", "label": 3},
    {"text": "How do I contact Project Management for support?", "label": 3},
    {"text": "I want to use Project Management services for my project.", "label": 3},

    # Customer Relationship Management
    {"text": "I want to avail the services of Customer Relationship Management.", "label": 4},
    {"text": "How can I access the CRM services?", "label": 4},
    {"text": "I need support from the Customer Relationship Management team.", "label": 4},
    {"text": "Can I get assistance from the CRM department?", "label": 4},
    {"text": "How do I use Customer Relationship Management services?", "label": 4},
    {"text": "I want to contact CRM for help.", "label": 4},
    {"text": "I need help from the Customer Relationship team.", "label": 4},
    {"text": "Can CRM assist me with customer queries?", "label": 4},
    {"text": "I want to access services from the Customer Relationship Management department.", "label": 4},
    {"text": "How do I avail CRM services for my project?", "label": 4},

    # Requesting Money for a project
    {"text": "How can I request funding for a new project?", "label": 5},
    {"text": "Can I submit a budget request for additional funds?", "label": 5},
    {"text": "What is the process for applying for project funding?", "label": 5},
    {"text": "How do I apply for project funds?", "label": 5},
    {"text": "Can I request more funds for my ongoing project?", "label": 5},
    {"text": "I need to request additional funding for my project.", "label": 5},
    {"text": "How do I make a request for project money?", "label": 5},
    {"text": "What is the process to ask for more funding?", "label": 5},
    {"text": "Can I get approval for extra project funding?", "label": 5},
    {"text": "How do I apply for funding for a new project?", "label": 5},

    # Performance check
    {"text": "How can I check my performance in the project?", "label": 6},
    {"text": "What is the process to check my project performance?", "label": 6},
    {"text": "How do I know if I'm meeting the project performance targets?", "label": 6},
    {"text": "Can I get a performance report for my current project?", "label": 6},
    {"text": "I want to know how my performance is being evaluated.", "label": 6},
    {"text": "How do I get feedback on my project performance?", "label": 6},
    {"text": "Can I track my performance throughout the project?", "label": 6},
    {"text": "How do I review my performance on the project?", "label": 6},
    {"text": "I want to see how my project performance is doing.", "label": 6},
    {"text": "Can I request a performance check on my ongoing project?", "label": 6},

    # Checking project details
    {"text": "How can I check the details of my project?", "label": 7},
    {"text": "Can I get the latest updates on my project?", "label": 7},
    {"text": "Where can I find the details of my ongoing project?", "label": 7},
    {"text": "I want to know the current status of my project.", "label": 7},
    {"text": "How do I check the progress of my project?", "label": 7},
    {"text": "Can I get a report on my project's current details?", "label": 7},
    {"text": "How do I access project details?", "label": 7},
    {"text": "Can I view the current status of my project?", "label": 7},
    {"text": "How can I check the progress and details of my project?", "label": 7},
    {"text": "What is the status of my project? I want to check the details.", "label": 7},

    # Scheduling Interviews
   {"text": "I would like to schedule an interview for the candidate regarding this project.", "label": 8},
  {"text": "Can you arrange an interview with the candidate for this project?", "label": 8},
  {"text": "Please help in scheduling an interview with the candidate for the project position.", "label": 8},
  {"text": "I need assistance to book an interview for the candidate in relation to this project.", "label": 8},
  {"text": "Can you coordinate with the candidate and schedule an interview for the project role?", "label": 8},
  {"text": "Please arrange a suitable time for the candidate’s interview for this project.", "label": 8},
  {"text": "I would appreciate it if you could schedule the candidate's interview for this project role.", "label": 8},
  {"text": "Let’s organize an interview with the candidate for the project position.", "label": 8},
  {"text": "Can you confirm the schedule for the candidate’s interview for this project?", "label": 8},
  {"text": "Please help in setting up an interview with the candidate for the project opportunity.", "label": 8},

    # Track the Inventory
    {"text": "I need to keep track of the inventory and stock to see if anything is running low.", "label": 9},
    {"text": "I want to monitor the inventory and stock levels to ensure nothing is understocked.", "label": 9},
    {"text": "I aim to track the stock and inventory to check if we have any low items.", "label": 9},
    {"text": "It’s important to keep an eye on the inventory so I can verify if any stock is low.", "label": 9},
    {"text": "I want to track inventory levels so I can check if we need to reorder anything.", "label": 9},
    {"text": "I need to monitor the stock to identify if any items are running low.", "label": 9},
    {"text": "I plan to track the inventory to make sure nothing is in short supply.", "label": 9},
    {"text": "I want to keep a check on the inventory and stock to ensure nothing is running low.", "label": 9},
    {"text": "I need to track the stock and inventory so I can quickly identify low levels.", "label": 9},
    {"text": "I want to track the inventory and stock to make sure we don’t run out of anything.", "label": 9},

    # CRM
    {"text": "What is the latest query from a client about this project, and has it been resolved?", "label": 10},
    {"text": "Can you update me on the most recent client query for this project and its resolution status?", "label": 10},
    {"text": "Please provide the latest client query for this project and let me know if it has been addressed.", "label": 10},
    {"text": "Could you let me know the latest issue raised by a client and if it has been resolved?", "label": 10},
    {"text": "What is the most recent client concern for this project, and is it resolved?", "label": 10},
    {"text": "Can you tell me the latest client query regarding the project and whether it has been resolved?", "label": 10},
    {"text": "What’s the current client query for the project, and has it been resolved yet?", "label": 10},
    {"text": "Please inform me about the latest client inquiry for this project and whether it’s been addressed.", "label": 10},
    {"text": "What is the recent query raised by the client, and has it been resolved yet?", "label": 10},
    {"text": "Could you share the latest client query for the project and confirm if it has been resolved?", "label": 10},
]

#Convert dataset into a Hugging Face Dataset format
dataset = Dataset.from_dict({
    "text": [item['text'] for item in dataset],
    "label": [item['label'] for item in dataset]
})

# Tokenizer and model for fine-tuning
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=11)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=16,
    num_train_epochs=30,
    weight_decay=0.01,
)

# Trainer for model training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)


trainer.train()




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,2.253122
2,No log,2.093638
3,No log,1.940993
4,No log,1.814503
5,No log,1.679529
6,No log,1.533929
7,No log,1.422485
8,No log,1.293944
9,No log,1.166526
10,No log,1.066445


TrainOutput(global_step=210, training_loss=0.9439262026832217, metrics={'train_runtime': 435.9984, 'train_samples_per_second': 7.569, 'train_steps_per_second': 0.482, 'total_flos': 868336645017600.0, 'train_loss': 0.9439262026832217, 'epoch': 30.0})